## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [ ]:
from pyspark.sql.functions import dense_rank,year,col,to_date,to_timestamp, when,date_add,date_format, expr, regexp_replace
from pyspark.sql.types import *

In [ ]:
# File location and type
# Source: https://www.kaggle.com/deepcontractor/top-video-games-19952021-metacritic
file_location = "/FileStore/tables/all_games.csv"
file_type = "csv"

schema = StructType([StructField("name", StringType(), True), StructField("platform", StringType(), True),StructField("release_date", DateType(), True), StructField("summary", StringType(), True), StructField("meta_score", IntegerType(), True),StructField("user_review", FloatType(), True)])

df = spark.read.option("inferSchema", "true").schema(schema).option("dateFormat", "dd.MM.YY").option("header", "true").csv(file_location)
df.cache()
df.show(5)
df.printSchema()

+--------------------+--------------+------------+--------------------+----------+-----------+
|                name|      platform|release_date|             summary|meta_score|user_review|
+--------------------+--------------+------------+--------------------+----------+-----------+
|The Legend of Zel...|   Nintendo 64|  1998-11-23|As a young boy, L...|        99|        9.1|
|Tony Hawk's Pro S...|   PlayStation|  2000-09-20|As most major pub...|        98|        7.4|
| Grand Theft Auto IV| PlayStation 3|  2008-04-29|"[Metacritic's 20...|      null|       null|
|         SoulCalibur|     Dreamcast|  1999-09-08|This is a tale of...|        98|        8.4|
| Grand Theft Auto IV|      Xbox 360|  2008-04-29|"[Metacritic's 20...|      null|       null|
+--------------------+--------------+------------+--------------------+----------+-----------+
only showing top 5 rows

root
 |-- name: string (nullable = true)
 |-- platform: string (nullable = true)
 |-- release_date: date (nullable = tru

##Brisanje pocetnog blanko karaktera u nazivu platforme

In [ ]:
df = df.withColumn("platform",expr("substr(platform, 2, length(platform)-1)"))
df.show()

+--------------------+-------------+------------+--------------------+----------+-----------+
|                name|     platform|release_date|             summary|meta_score|user_review|
+--------------------+-------------+------------+--------------------+----------+-----------+
|The Legend of Zel...|  Nintendo 64|  1998-11-23|As a young boy, L...|        99|        9.1|
|Tony Hawk's Pro S...|  PlayStation|  2000-09-20|As most major pub...|        98|        7.4|
| Grand Theft Auto IV|PlayStation 3|  2008-04-29|"[Metacritic's 20...|      null|       null|
|         SoulCalibur|    Dreamcast|  1999-09-08|This is a tale of...|        98|        8.4|
| Grand Theft Auto IV|     Xbox 360|  2008-04-29|"[Metacritic's 20...|      null|       null|
|  Super Mario Galaxy|          Wii|  2007-11-12|[Metacritic's 200...|        97|        9.1|
|Super Mario Galaxy 2|          Wii|  2010-05-23|Super Mario Galax...|        97|        9.1|
|Red Dead Redempti...|     Xbox One|  2018-10-26|Developed b

In [ ]:
dbutils.fs.rm('dbfs:/user/hive/warehouse/df', True)
df.createOrReplaceTempView("df")
df.write.mode("overwrite").format("parquet").saveAsTable("df")

##Konvertovanje datuma u format mesec godina

In [ ]:
spark.conf.set("spark.sql.legacy.timeParserPolicy","LEGACY")
title_year = df.select("name","platform", date_format("release_date", "M YYYY").alias ("date")).where(year("release_date") >= 2016).orderBy("release_date")

##Parsiranje godine i castovanje u int

In [ ]:
title_year=title_year.withColumn("year",expr("substr(date, 2, 5)").cast('int'))
title_year.show()

+--------------------+-------------+------+----+
|                name|     platform|  date|year|
+--------------------+-------------+------+----+
|         Pony Island|           PC|1 2016|2016|
|    Hardware: Rivals|           PC|1 2016|2016|
|           Amplitude|PlayStation 4|1 2016|2016|
|       Lovely Planet|     Xbox One|1 2016|2016|
|          Punch Club|           PC|1 2016|2016|
| That Dragon, Cancer|           PC|1 2016|2016|
|             Tharsis|           PC|1 2016|2016|
|Gone Home: Consol...|PlayStation 4|1 2016|2016|
|             Tharsis|PlayStation 4|1 2016|2016|
|Star Wars Pinball...|PlayStation 4|1 2016|2016|
|     The Banner Saga|PlayStation 4|1 2016|2016|
|Assassin's Creed ...|           PC|1 2016|2016|
|Assassin's Creed ...|     Xbox One|1 2016|2016|
|     The Banner Saga|     Xbox One|1 2016|2016|
|Assassin's Creed ...|PlayStation 4|1 2016|2016|
|            Oxenfree|           PC|1 2016|2016|
|  Echoes Of Aetheria|           PC|1 2016|2016|
|            Oxenfre

In [ ]:
dbutils.fs.rm('dbfs:/user/hive/warehouse/title_year', True)
title_year.createOrReplaceTempView("title_year")
title_year.write.mode("overwrite").format("parquet").saveAsTable("title_year")


##Ucitavanje podataka o koroni

In [ ]:
# File location and type
# Source: https://www.kaggle.com/georgesaavedra/covid19-dataset
# Kolone: iso_code, continent, location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths

file_location2 = "/FileStore/tables/owid_covid_data.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df2 = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .option("dateFormat", "yyyy-MM-dd") \
  .load(file_location2)

df2.printSchema()

root
 |-- iso_code: string (nullable = true)
 |-- continent: string (nullable = true)
 |-- location: string (nullable = true)
 |-- date: string (nullable = true)
 |-- total_cases: double (nullable = true)
 |-- new_cases: double (nullable = true)
 |-- new_cases_smoothed: double (nullable = true)
 |-- total_deaths: double (nullable = true)
 |-- new_deaths: double (nullable = true)
 |-- new_deaths_smoothed: double (nullable = true)
 |-- total_cases_per_million: double (nullable = true)
 |-- new_cases_per_million: double (nullable = true)
 |-- new_cases_smoothed_per_million: double (nullable = true)
 |-- total_deaths_per_million: double (nullable = true)
 |-- new_deaths_per_million: double (nullable = true)
 |-- new_deaths_smoothed_per_million: double (nullable = true)
 |-- reproduction_rate: double (nullable = true)
 |-- icu_patients: double (nullable = true)
 |-- icu_patients_per_million: double (nullable = true)
 |-- hosp_patients: double (nullable = true)
 |-- hosp_patients_per_million

##Konvertovanje stringa u date

In [ ]:
new_corona_set=df2.withColumn("time", to_date(col("date"), "yyyy-MM-dd")).drop("date")

dbutils.fs.rm('dbfs:/user/hive/warehouse/new_corona_set', True)
new_corona_set.createOrReplaceTempView("new_corona_set")
new_corona_set.write.mode("overwrite").format("parquet").saveAsTable("new_corona_set")


In [ ]:
dbutils.fs.rm('dbfs:/user/hive/warehouse/df', True)
df.createOrReplaceTempView("df")
df.write.mode("overwrite").format("parquet").saveAsTable("df")

##Prebacivanje blanko karaktera u _

In [ ]:

df = df.withColumn("platform", expr("replace(platform,' ','_')"))
title_year2 = df.select("name","platform", date_format("release_date", "M YYYY").alias ("date")).where(year("release_date") >= 2021)
title_year2.show()


title_year2.cache()



+--------------------+-------------+-------+
|                name|     platform|   date|
+--------------------+-------------+-------+
|Disco Elysium: Th...|           PC| 3 2021|
|The House in Fata...|       Switch| 4 2021|
|Tetris Effect: Co...|       Switch|10 2021|
|               Hades|Xbox_Series_X| 8 2021|
|               Hades|PlayStation_5| 8 2021|
|       Psychonauts 2|     Xbox_One| 8 2021|
|Microsoft Flight ...|Xbox_Series_X| 7 2021|
|Tony Hawk's Pro S...|PlayStation_5| 3 2021|
|Chicory: A Colorf...|           PC| 6 2021|
|Mass Effect Legen...|     Xbox_One| 5 2021|
|        It Takes Two|PlayStation_4| 3 2021|
|       Metroid Dread|       Switch|10 2021|
|        It Takes Two|Xbox_Series_X| 3 2021|
|Super Mario 3D Wo...|       Switch| 2 2021|
|Final Fantasy VII...|PlayStation_5| 6 2021|
|Tony Hawk's Pro S...|Xbox_Series_X| 3 2021|
|        Synth Riders|PlayStation_4| 8 2021|
|       Psychonauts 2|           PC| 8 2021|
|Disco Elysium: Th...|PlayStation_5| 3 2021|
|         

In [ ]:
dbutils.fs.rm('dbfs:/user/hive/warehouse/title_year2', True)
title_year2.createOrReplaceTempView("title_year2")
title_year2.write.mode("overwrite").format("parquet").saveAsTable("title_year2")

## Transformacija PlayStation u PS radi boljeg pregleda

In [ ]:
df = df.withColumn('platform', regexp_replace('platform', 'PlayStation', 'PS'))
df.show(1000)

+--------------------+----------------+------------+--------------------+----------+-----------+----------------+
|                name|        platform|release_date|             summary|meta_score|user_review|              ps|
+--------------------+----------------+------------+--------------------+----------+-----------+----------------+
|The Legend of Zel...|     Nintendo_64|  1998-11-23|As a young boy, L...|        99|        9.1|     Nintendo_64|
|Tony Hawk's Pro S...|              PS|  2000-09-20|As most major pub...|        98|        7.4|              PS|
| Grand Theft Auto IV|            PS_3|  2008-04-29|"[Metacritic's 20...|      null|       null|            PS_3|
|         SoulCalibur|       Dreamcast|  1999-09-08|This is a tale of...|        98|        8.4|       Dreamcast|
| Grand Theft Auto IV|        Xbox_360|  2008-04-29|"[Metacritic's 20...|      null|       null|        Xbox_360|
|  Super Mario Galaxy|             Wii|  2007-11-12|[Metacritic's 200...|        97|    